## DOG & CAT TL(MobileNetV2) with Fine-Tuning (92% accuracy)

![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)


In [1]:
!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

--2024-02-01 19:46:10--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.128.207, 142.251.6.207, 172.217.214.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.128.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M   164MB/s    in 0.4s    

2024-02-01 19:46:10 (164 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [2]:
import tensorflow as tf
tf.__version__

'2.15.0'

In [3]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [4]:
#unzip folder
dataset_path = "./cats_and_dogs_filtered.zip"
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
zip_object.extractall("./")
zip_object.close()

In [5]:
dataset_path_new = "./cats_and_dogs_filtered/"

train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Building the model

### Loading the pre-trained model (MobileNetV2)

MobileNetV2 is a convolutional neural network architecture that seeks to perform well on mobile devices. It is based on an inverted residual structure where the residual connections are between the bottleneck layers. The intermediate expansion layer uses lightweight depthwise convolutions to filter features as a source of non-linearity. As a whole, the architecture of MobileNetV2 contains the initial fully convolution layer with 32 filters, followed by 19 residual bottleneck layers




<img src= "https://frenzy86.s3.eu-west-2.amazonaws.com/python/mobilenetv2.png" width=500>


For example: MobileNet (architecture that we use) supports: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [6]:
IMG_SHAPE = (224, 224, 3)

In [7]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights="imagenet"
                                               )

9406464/9406464 [==============================] - 0s 0us/step


In [8]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[

### Freezing the base model

In [9]:
base_model.trainable = False

### Defining the custom head for our network

In [10]:
base_model.output

<KerasTensor: shape=(None, 7, 7, 1280) dtype=float32 (created by layer 'out_relu')>

In [11]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [12]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [13]:
prediction_layer = tf.keras.layers.Dense(units=2,
                                         activation='softmax'
                                         )(global_average_layer)

### Defining the model

In [14]:
model = tf.keras.models.Model(inputs=base_model.input,
                              outputs=prediction_layer
                              )

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[0][0]']        

### Compiling the model

In [16]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss="categorical_crossentropy", #2 neurons
              metrics=["accuracy"]
              )

### Creating Data Generators

Resizing images

    Big pre-trained architecture support only certain input sizes.

For example: MobileNet (architecture that we use) supports: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [17]:
image_size = (IMG_SHAPE[0], IMG_SHAPE[1])
batch_size = 64

In [18]:
datagenerator  = ImageDataGenerator(
                                    rescale=1/255.,
                                    #rotation_range=20,
                                    #width_shift_range=0.1,
                                    #height_shift_range=0.1,
                                    #shear_range=0.1,
                                    #zoom_range=0.2,
                                    #horizontal_flip=True,
                                    )

In [19]:
training_set = datagenerator.flow_from_directory(train_dir,
                                                seed=667,
                                                target_size=image_size,
                                                batch_size=batch_size,
                                                class_mode = 'categorical',
                                                shuffle=False
                                                 )

Found 2000 images belonging to 2 classes.


In [20]:
validation_set  = datagenerator.flow_from_directory(validation_dir,
                                                    seed=667,
                                                    target_size=image_size,
                                                    batch_size=batch_size,
                                                    class_mode = 'categorical',
                                                    shuffle=False
                                                    )

Found 1000 images belonging to 2 classes.


### Training the model

In [21]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
file_path="weights_best.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_accuracy", mode="max", patience=5)
callbacks_list = [checkpoint, early] #early

history = model.fit(training_set,
                    validation_data=validation_set,
                    epochs=5,
                    steps_per_epoch=len(training_set),
                    validation_steps=len(validation_set),
                    callbacks=callbacks_list
                    )

Epoch 1/5
32/32 [==============================] - ETA: 0s - loss: 1.0476 - accuracy: 0.3585
Epoch 1: val_accuracy improved from -inf to 0.39400, saving model to weights_best.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


32/32 [==============================] - 17s 350ms/step - loss: 1.0476 - accuracy: 0.3585 - val_loss: 0.9056 - val_accuracy: 0.3940
Epoch 2/5
32/32 [==============================] - ETA: 0s - loss: 0.8324 - accuracy: 0.4685
Epoch 2: val_accuracy improved from 0.39400 to 0.53200, saving model to weights_best.h5
32/32 [==============================] - 9s 276ms/step - loss: 0.8324 - accuracy: 0.4685 - val_loss: 0.7553 - val_accuracy: 0.5320
Epoch 3/5
32/32 [==============================] - ETA: 0s - loss: 0.6882 - accuracy: 0.6035
Epoch 3: val_accuracy improved from 0.53200 to 0.69500, saving model to weights_best.h5
32/32 [==============================] - 9s 285ms/step - loss: 0.6882 - accuracy: 0.6035 - val_loss: 0.6050 - val_accuracy: 0.6950
Epoch 4/5
32/32 [==============================] - ETA: 0s - loss: 0.5553 - accuracy: 0.7330
Epoch 4: val_accuracy improved from 0.69500 to 0.77900, saving model to weights_best.h5
32/32 [==============================] - 9s 282ms/step - loss: 

### Transfer learning model evaluation

In [22]:
valid_loss, valid_accuracy = model.evaluate(validation_set)

16/16 [==============================] - 3s 171ms/step - loss: 0.4080 - accuracy: 0.8500


In [23]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.8500000238418579


## Fine tuning


There are a few pointers:

- DO NOT use Fine tuning on the whole network; only a few top layers are enough. In most cases, they are more specialized. The goal of the Fine-tuning is to adopt that specific part of the network for our custom (new) dataset.
- Start with the fine tunning AFTER you have finished with transfer learning step. If we try to perform Fine tuning immediately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model.



<img src= "https://frenzy86.s3.eu-west-2.amazonaws.com/python/finetuning.png" width=1000>

### Un-freeze a few top layers from the model

In [ ]:
base_model.trainable = True

In [ ]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 154


In [ ]:
## 154 layers-- we freeze the first 80 for Fine Tuning

In [24]:
#Fine tuning threshold

fine_tune_at = 80

In [25]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compiling the model for fine-tuning

In [26]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

### Fine tuning

In [27]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
file_path="weights_best.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_accuracy", mode="max", patience=5)
callbacks_list = [checkpoint, early] #early

history = model.fit(training_set,
                    validation_data=validation_set,
                    epochs=5,
                    steps_per_epoch=len(training_set),
                    validation_steps=len(validation_set),
                    callbacks=callbacks_list
                    )

Epoch 1/5
32/32 [==============================] - ETA: 0s - loss: 0.3808 - accuracy: 0.8635
Epoch 1: val_accuracy improved from -inf to 0.89200, saving model to weights_best.h5
32/32 [==============================] - 12s 310ms/step - loss: 0.3808 - accuracy: 0.8635 - val_loss: 0.3364 - val_accuracy: 0.8920
Epoch 2/5
32/32 [==============================] - ETA: 0s - loss: 0.3124 - accuracy: 0.8990
Epoch 2: val_accuracy improved from 0.89200 to 0.90600, saving model to weights_best.h5
32/32 [==============================] - 9s 275ms/step - loss: 0.3124 - accuracy: 0.8990 - val_loss: 0.2879 - val_accuracy: 0.9060
Epoch 3/5
32/32 [==============================] - ETA: 0s - loss: 0.2731 - accuracy: 0.9170
Epoch 3: val_accuracy improved from 0.90600 to 0.92300, saving model to weights_best.h5
32/32 [==============================] - 9s 270ms/step - loss: 0.2731 - accuracy: 0.9170 - val_loss: 0.2483 - val_accuracy: 0.9230
Epoch 4/5
32/32 [==============================] - ETA: 0s - loss:

### Evaluating the fine tuned model

In [28]:
valid_loss, valid_accuracy = model.evaluate(validation_set)

16/16 [==============================] - 3s 194ms/step - loss: 0.1945 - accuracy: 0.9420


In [29]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9419999718666077
